Лабораторная №6
1. Развернуть локально СУБД, описать структуру ее файлов в файловой системе. Настроить подключение через инструменты администрирования БД
2. Спроектировать хранилище. Нормализовать до необходимой НФ. Создать БД на основе ""проектного плана"" в развернутой СУБД (пункт 1). Уделить внимание ограничениям, ключам, типам данных
3. Создать генератор заполнения спроектированной БД. Заполнить псевдореальными данными из генератора. Уделить внимания связям в данных. Создать объекты для отображения типов связей между таблицами
4. Произвести администрирование групповых политик в БД. Произвести защиту БД от стороннних подключений. 
5. Создать view / mateialized view с ""отчетом"" по данным из таблиц БД. Отчет должен содержать как минимум 1 JOIN а также некоторые вычисления. 

## 1. Хранение сервера
Один сервер обслуживает кластер баз данных

Можно подключиться сразу к нескольким базам данных, но нельзя получить объекты одной БД из другой

При инициализации кластера создаются три БД 
- template1 
- template0 
- postgres
  
Каждая новая БД создается из шаблона - по умолчанию template1
### Описание файлов в файловой системе
- base 
	- Хранение каталогов с базами данных: по каталогу  на БД
		- Файлы, соответствующие объектам БД
- global - каталог с глобальными таблицами (ex. pg_database)
- log - журнал работы сервера
- pg_xact - информация о статусах транзакций, по  2 бита на транзакцию
- pg_stat  - статистика работы сервера
- pg_tblspc - ссылки на табличные пространства
- ...

## 2. Хранилище

Описание таблиц
- films
    
    Хранит информацию о фильмах (название, описание ...)

    director_id - ссылка режиссера из таблицы person (Каждому фильму соответвтует один режиссер)
- genre
    
    Жанры фильмов

    Каждому фильму может соответствовать несколько жанров, поэтому была создана промежуточная таблица film_genre

- person

     Таблица, содежащая информацию о создателях фильма (актеры режиссеры)
     Для поиска всех людей, работавших на созданием создана таблица film_person

- film_list
    
    Подборки фильмов, для добавления фильма в подборку есть таблица fl_films

- Users
     
     Таблица с пользователями 
- raiting
    
    Таблица содержащая оценки и отзывы пользователей, после удаления пользователя его отзыв должен остаться, поэтому вместо составного первичного ключа используется колонка guid

Пользователи могут отмечать фильмы как просмотренные и отложенные для просмотра, для этого была создана таблица 
- user_lists с партиционированием по название (Watch Later, Watched)


Все таблицы находятся в 3НФ

![Films.png](images/Films.png)

## 3. Генератор заполнения


- Таблицы Films, Genres, Person заполним из файла `imdb_to_1000.csv`, взятого с Kaggle
- Оставшиеся таблицы заполним с помощью библиотеки Faker

In [52]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from faker import Faker
import uuid
import random
from random import randint

In [53]:
host = 'localhost:5435'
user = 'admin'
database_name = 'filmsdb2'
password = 'admin'
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}/{database_name}')

In [54]:
query = f'''SELECT tablename, tableowner
            FROM pg_tables
            where tableowner = 'admin'
         '''
df_res = pd.read_sql(query, con=engine)
df_res

,tablename,tableowner
0,user_lists,admin
1,film_rating,admin
2,genres,admin
3,film_lists,admin
4,pg_statistic,admin
...,...,...
77,pg_largeobject,admin
78,sql_parts,admin
79,sql_features,admin
80,sql_implementation_info,admin


In [55]:
df = pd.read_csv('imdb_top_1000.csv')
df.head()

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


In [56]:
df = df.drop(['Poster_Link', 'Certificate', 'No_of_Votes', 'Meta_score', 'IMDB_Rating', 'Gross'], axis=1)

In [57]:
df.isna().sum()

Series_Title     0
Released_Year    0
Runtime          0
Genre            0
Overview         0
Director         0
Star1            0
Star2            0
Star3            0
Star4            0
dtype: int64

In [58]:
df['Released_Year'].unique()

array(['1994', '1972', '2008', '1974', '1957', '2003', '1993', '2010',
       '1999', '2001', '1966', '2002', '1990', '1980', '1975', '2020',
       '2019', '2014', '1998', '1997', '1995', '1991', '1977', '1962',
       '1954', '1946', '2011', '2006', '2000', '1988', '1985', '1968',
       '1960', '1942', '1936', '1931', '2018', '2017', '2016', '2012',
       '2009', '2007', '1984', '1981', '1979', '1971', '1963', '1964',
       '1950', '1940', '2013', '2005', '2004', '1992', '1987', '1986',
       '1983', '1976', '1973', '1965', '1959', '1958', '1952', '1948',
       '1944', '1941', '1927', '1921', '2015', '1996', '1989', '1978',
       '1961', '1955', '1953', '1925', '1924', '1982', '1967', '1951',
       '1949', '1939', '1937', '1934', '1928', '1926', '1920', '1970',
       '1969', '1956', '1947', '1945', '1930', '1938', '1935', '1933',
       '1932', '1922', '1943', 'PG'], dtype=object)

In [59]:
a = df[df['Released_Year'] == 'PG']['Released_Year'].index
df.iloc[a, 1] = '1995'

In [60]:
fake = Faker()
Faker.seed(777)

In [61]:
n = df.shape[0]
films_uuid = [uuid.uuid4() for i in range(n)]
mpaa = random.choices(['G', 'PG', 'PG-13', 'R', 'NC-17'], k=n)
runtime = [df['Runtime'][i].split()[0] for i in range(n)]
person = {}
genre = {}
director_id = []
film_crew = []
film_genre = []

for i in range(n):
    for j in df['Genre'][i].split(', '):
        guid = genre.setdefault(j, uuid.uuid4())
        film_genre.append([
            films_uuid[i],
            genre.setdefault(j, uuid.uuid4())
        ])

    act = set(df.iloc[i, 6:10])
    for j in act:
        film_crew.append([
            films_uuid[i],
            person.setdefault(j, uuid.uuid4()),
            'Actor'
        ])

    guid = person.setdefault(df['Director'][i], uuid.uuid4())
    film_crew.append([films_uuid[i], guid, 'Director'])
    director_id.append(guid)


In [62]:
birth_date = [fake.date_of_birth(minimum_age=24, maximum_age=85) for i in range(len(person))]
first_name = []
last_name = []
person_uuid = []
for k, v in person.items():
    person_uuid.append(v)
    first_name.append(k.split()[0])
    last_name.append(' '.join(k.split()[1:]))

persons = pd.DataFrame({
    'guid': person_uuid,
    'first_name': first_name,
    'last_name': last_name,
    'birth_date': birth_date
})
persons

,guid,first_name,last_name,birth_date
0,9cadaadc-bba4-4619-9f51-45f66d3ba847,William,Sadler,1955-06-22
1,36343105-c987-4f22-a9c4-8a4c0c12473e,Morgan,Freeman,1970-03-16
2,9ed95cc3-df14-421f-836d-1d9c0701960c,Tim,Robbins,1970-03-10
3,1d37d0bd-e67b-48eb-a53c-e93d3a82a2ce,Bob,Gunton,1964-12-12
4,949853a8-085f-49c6-8d47-c3e5ab5766a9,Frank,Darabont,1979-01-11
...,...,...,...,...
3197,7b04963d-4fbf-47be-acc3-ed1353440c6b,Tallulah,Bankhead,1989-09-21
3198,2244e4fa-d7dc-4542-af21-d0524ee4e0de,Madeleine,Carroll,1956-09-27
3199,9ca1bab5-f9a1-4950-be74-ab3ca70386d1,Robert,Donat,1941-06-26
3200,6bcd81ff-f08a-44c4-bec9-942532bb6cf3,Godfrey,Tearle,1976-10-03


In [63]:
persons.to_sql('persons', engine, if_exists='append', index=False)

202

In [64]:
genre = np.array([[k, v] for k, v in genre.items()])
genres = pd.DataFrame({
    'guid': genre[:, 1],
    'genre': genre[:, 0]
})
genres

,guid,genre
0,0776b373-567a-44dc-954a-03e6c032ba18,Drama
1,8149bc43-5232-477d-957f-c36a986be009,Crime
2,72ccc4c8-0be1-4932-942b-4bc64e11ad97,Action
3,350f9b33-da16-472f-bb52-958223eee985,Adventure
4,663ef8fc-d085-4a4e-a721-9b81d6771052,Biography
5,ed21d623-9491-44df-9ea3-baaa4381220a,History
6,0f7e67aa-bbd4-4e97-b21f-4663fa873c7e,Sci-Fi
7,99067a52-bf57-41e8-93e1-5d88a4b5e82a,Romance
8,59d88398-1a2c-4c2e-980d-95b9ae98d70a,Western
9,c2e5e1e2-34a1-4538-96b4-05e276acdda9,Fantasy


In [65]:
genres.to_sql('genres', engine, if_exists='append', index=False)

21

In [66]:
films = pd.DataFrame({'guid': films_uuid,
                      'title': df['Series_Title'],
                      'year_released': df['Released_Year'],
                      'rating_mpaa': mpaa,
                      'director_id': director_id,
                      'runtime': runtime,
                      'summary': df['Overview']})

films

,guid,title,year_released,rating_mpaa,director_id,runtime,summary
0,059030b8-4537-43f1-90c4-a67968b0a82a,The Shawshank Redemption,1994,R,949853a8-085f-49c6-8d47-c3e5ab5766a9,142,Two imprisoned men bond over a number of years...
1,e953357a-3dc1-4dd2-bdc7-5ace02749bd6,The Godfather,1972,PG,cfa3c7d7-693f-4602-ae43-a815f31652cc,175,An organized crime dynasty's aging patriarch t...
2,8206ad55-c851-4331-9c23-df0c1fcecac3,The Dark Knight,2008,G,600853fb-3d6a-4516-9ace-8c3f7785879e,152,When the menace known as the Joker wreaks havo...
3,47d3971d-38f0-4bdc-b3ec-6cb5bad0f4f5,The Godfather: Part II,1974,PG-13,cfa3c7d7-693f-4602-ae43-a815f31652cc,202,The early life and career of Vito Corleone in ...
4,51eeb3f6-4ada-4025-9fcc-fba2577a2f76,12 Angry Men,1957,R,04ad6685-b25c-4b4f-8de6-6be390389c9a,96,A jury holdout attempts to prevent a miscarria...
...,...,...,...,...,...,...,...
995,dc0e51cb-742e-425d-bad7-5edfad7a0530,Breakfast at Tiffany's,1961,NC-17,e1cd7c1c-9928-477c-9709-442f9f4bc77e,115,A young New York socialite becomes interested ...
996,48c1ef15-af3e-4027-90ad-f74d10620fed,Giant,1956,PG,1c5e0e39-1e40-4bbf-b52e-3363ae130ecd,201,Sprawling epic covering the life of a Texas ca...
997,e049ff31-729a-466f-ae27-7eaa0afac75f,From Here to Eternity,1953,NC-17,4b0f340a-32a7-4312-b74f-25a7c61a6426,118,"In Hawaii in 1941, a private is cruelly punish..."
998,3dd0540a-b870-45fe-8acc-7915109d83b2,Lifeboat,1944,R,100b980f-7a5b-4e51-a699-bff0cc02ebf8,97,Several survivors of a torpedoed merchant ship...


In [67]:
films.to_sql('films', engine, if_exists='append', index=False)

1000

In [68]:
film_person = np.array(film_crew)
film_genre = np.array(film_genre)
films_genre = pd.DataFrame({'film_guid': film_genre[:, 0],
                            'genre_guid': film_genre[:, 1]})
films_person = pd.DataFrame({'guid': [uuid.uuid4() for i in range(len(film_person))],
                             'film_guid': film_person[:, 0],
                             'person_guid': film_person[:, 1],
                             'profession': film_person[:, 2]})
films_genre

,film_guid,genre_guid
0,059030b8-4537-43f1-90c4-a67968b0a82a,0776b373-567a-44dc-954a-03e6c032ba18
1,e953357a-3dc1-4dd2-bdc7-5ace02749bd6,8149bc43-5232-477d-957f-c36a986be009
2,e953357a-3dc1-4dd2-bdc7-5ace02749bd6,0776b373-567a-44dc-954a-03e6c032ba18
3,8206ad55-c851-4331-9c23-df0c1fcecac3,72ccc4c8-0be1-4932-942b-4bc64e11ad97
4,8206ad55-c851-4331-9c23-df0c1fcecac3,8149bc43-5232-477d-957f-c36a986be009
...,...,...
2536,3dd0540a-b870-45fe-8acc-7915109d83b2,0776b373-567a-44dc-954a-03e6c032ba18
2537,3dd0540a-b870-45fe-8acc-7915109d83b2,2f8a32c5-7bfe-44e3-ae05-43dfa16ccc54
2538,d709e286-a04b-4f89-9ad0-439656e96546,8149bc43-5232-477d-957f-c36a986be009
2539,d709e286-a04b-4f89-9ad0-439656e96546,43e2ecef-a432-47de-a3e8-aeeabb7ab34f


In [69]:
films_genre.to_sql('films_genre', engine, if_exists='append', index=False)

541

In [70]:
films_person

,guid,film_guid,person_guid,profession
0,74feac38-5dc9-4062-b1a6-5b2ef9f03c84,059030b8-4537-43f1-90c4-a67968b0a82a,9cadaadc-bba4-4619-9f51-45f66d3ba847,Actor
1,b55555af-d365-4bea-8d18-6a3c34a0581a,059030b8-4537-43f1-90c4-a67968b0a82a,36343105-c987-4f22-a9c4-8a4c0c12473e,Actor
2,0b842298-a2e3-4871-82aa-40731efa7030,059030b8-4537-43f1-90c4-a67968b0a82a,9ed95cc3-df14-421f-836d-1d9c0701960c,Actor
3,3a0231f9-b0d0-43d8-8299-9598e25dfbb1,059030b8-4537-43f1-90c4-a67968b0a82a,1d37d0bd-e67b-48eb-a53c-e93d3a82a2ce,Actor
4,46652032-d82e-4568-bf24-b55c9080f9e9,059030b8-4537-43f1-90c4-a67968b0a82a,949853a8-085f-49c6-8d47-c3e5ab5766a9,Director
...,...,...,...,...
4991,db3f4107-c62f-4ad7-874c-2e7a923c732b,d709e286-a04b-4f89-9ad0-439656e96546,2244e4fa-d7dc-4542-af21-d0524ee4e0de,Actor
4992,979eb4aa-57c2-4824-a291-87970a751866,d709e286-a04b-4f89-9ad0-439656e96546,9ca1bab5-f9a1-4950-be74-ab3ca70386d1,Actor
4993,d8361e08-a042-468d-85ec-193cc7987d2b,d709e286-a04b-4f89-9ad0-439656e96546,6bcd81ff-f08a-44c4-bec9-942532bb6cf3,Actor
4994,7fed57f3-c7c0-41b6-b0d8-af0da5d81024,d709e286-a04b-4f89-9ad0-439656e96546,d697bc70-db61-4d8f-9193-420e6b510b22,Actor


In [71]:
films_person.to_sql('film_person', engine, if_exists='append', index=False)

996

In [72]:
cnt_users = 1000
user_guid = [uuid.uuid4() for i in range(cnt_users)]
username = [fake.unique.user_name() for i in range(cnt_users)]
password = [fake.password() for i in range(cnt_users)]
email = [fake.unique.email() for i in range(cnt_users)]
birth_date = [fake.date_of_birth(minimum_age=14, maximum_age=70) for i in range(cnt_users)]
users = pd.DataFrame({
    'guid': user_guid,
    'username': username,
    'password': password,
    'email': email,
    'birth_date': birth_date
})
users

,guid,username,password,email,birth_date
0,099e57ea-7c47-4a36-8eea-550efb9dce9b,clarkemily,(H9&Aemr)a,randy25@example.net,1959-06-01
1,cf3c0c54-7e6a-4633-95fd-6608d30cfbf6,jonesthomas,D(BI4IfgYD,pdavis@example.net,1968-12-30
2,a7b0d6b1-f1bb-4ac2-ba37-f650480b7967,nguyenkevin,)zjSHchHi9,vhorton@example.net,1962-11-07
3,524c9785-45cc-4dd5-a71c-48c2c2ff1068,ufarrell,%t9)hRne&s,scott02@example.net,1990-07-23
4,d2e26ebe-103c-4626-bac3-59a9f358fba2,urios,ntxMGkml!0,martinezdavid@example.org,2009-06-22
...,...,...,...,...,...
995,98e0da67-31cd-4905-8e19-157ade5a1342,fmartinez,2&O2FNOvBz,brianjimenez@example.com,1976-08-25
996,4aea8f74-4a4e-46be-a403-0a5063d3bd11,philiphenson,Q9W8w!Tn(S,ynorris@example.net,1967-12-22
997,0ef49957-1d0c-41e4-aab8-a01f042f8339,josewilliams,_0EHKHOFml,gmontgomery@example.org,2007-08-27
998,3d0a5f5c-29e0-4e85-ab7c-ad5221e0c225,kchristensen,@PHDxEQ1#8,michael36@example.org,2003-02-09


In [73]:
users.to_sql('users', engine, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "users_username_key"
DETAIL:  Key (username)=(mjones) already exists.

[SQL: INSERT INTO users (guid, username, password, email, birth_date) VALUES (%(guid__0)s, %(username__0)s, %(password__0)s, %(email__0)s, %(birth_date__0)s), (%(guid__1)s, %(username__1)s, %(password__1)s, %(email__1)s, %(birth_date__1)s), (%(guid__2)s, % ... 91169 characters truncated ... __998)s), (%(guid__999)s, %(username__999)s, %(password__999)s, %(email__999)s, %(birth_date__999)s)]
[parameters: {'birth_date__0': datetime.date(1959, 6, 1), 'email__0': 'randy25@example.net', 'guid__0': UUID('099e57ea-7c47-4a36-8eea-550efb9dce9b'), 'username__0': 'clarkemily', 'password__0': '(H9&Aemr)a', 'birth_date__1': datetime.date(1968, 12, 30), 'email__1': 'pdavis@example.net', 'guid__1': UUID('cf3c0c54-7e6a-4633-95fd-6608d30cfbf6'), 'username__1': 'jonesthomas', 'password__1': 'D(BI4IfgYD', 'birth_date__2': datetime.date(196

In [ ]:
lst_guid = []
lst_title = []
films_ref = []
user_ref = []
for i in user_guid:
    n = randint(15, 100)
    m = randint(5, 50)
    user_ref += [i] * (n + m)
    lst_title += ['Viewed'] * n + ['Watch Later'] * m
    films_ref += random.sample(films_uuid, k=n) + random.sample(films_uuid, k=m)
user_lst = pd.DataFrame({
    'user_guid': user_ref,
    'film_guid': films_ref,
    'title': lst_title
})
user_lst

,user_guid,film_guid,title
0,84db3d2f-17b5-4450-8669-599cf8c0a7b6,27fad05c-5c2f-4275-b8e8-a43a73809c6a,Viewed
1,84db3d2f-17b5-4450-8669-599cf8c0a7b6,99a5f8e4-b630-42d0-b2e7-6312ad8d43c3,Viewed
2,84db3d2f-17b5-4450-8669-599cf8c0a7b6,ca4a4fde-5e0d-4944-97bc-c525998512bb,Viewed
3,84db3d2f-17b5-4450-8669-599cf8c0a7b6,354bdb2f-5a7b-453a-b127-599b665018a1,Viewed
4,84db3d2f-17b5-4450-8669-599cf8c0a7b6,198260b5-0933-4949-8d2f-d70de8148a36,Viewed
...,...,...,...
84486,f86969bf-680f-4d90-a56c-1f2370394e2f,b9dad22d-dce8-4fa5-9904-b4644495c13d,Watch Later
84487,f86969bf-680f-4d90-a56c-1f2370394e2f,6a321633-f992-4dca-934a-31fdee9dccad,Watch Later
84488,f86969bf-680f-4d90-a56c-1f2370394e2f,ac49ea8c-e7d5-4abd-82c9-c7f3243c4ed9,Watch Later
84489,f86969bf-680f-4d90-a56c-1f2370394e2f,dab14998-3b75-4a74-9c80-6efbe27097cd,Watch Later


In [ ]:
user_lst.to_sql('user_lists', engine, if_exists='append', index=False)

491

In [ ]:
rates = []
user_ref = []
films_ref = []
score = [i for i in range(1, 11)]
for i in films_uuid:
    n = randint(100, 250)
    films_ref += [i] * n
    rates += random.choices(score, k=n)
    user_ref += random.sample(user_guid, k=n)
reviews = [fake.paragraph(nb_sentences=3) for i in range(len(rates))]
rating_date = [fake.date_time_between_dates(datetime_start='-2y', datetime_end='now') for i in range(len(rates))]
rating_uuid = [uuid.uuid4() for i in range(len(rates))]


In [ ]:
raiting = pd.DataFrame({
    'guid': rating_uuid,
    'film_guid': films_ref,
    'user_guid': user_ref,
    'rating': rates,
    'rating_date': rating_date,
    'review': reviews
})
raiting

,guid,film_guid,user_guid,rating,rating_date,review
0,ca8baff9-8c71-43bd-9120-b0ff85078167,cbac3e24-e95f-4e1f-8e2e-4031ea902cd0,d78b69a6-a2ed-473c-af5c-bdda874dfea8,7,2025-08-30 06:25:27,Order test along. Exist trade central service ...
1,3b0ce4e5-51ad-4264-b67e-81e53f395339,cbac3e24-e95f-4e1f-8e2e-4031ea902cd0,bfd715d5-90cf-4c52-9c82-b0cb0bb8e14a,2,2024-01-28 06:13:20,Deep strategy of themselves parent card coach.
2,6b4f3ea3-d8b5-4110-9620-dd6ea5e92479,cbac3e24-e95f-4e1f-8e2e-4031ea902cd0,82bc2fa5-13ed-4dcb-88d1-cc53044635e0,3,2024-02-09 16:50:41,End behavior available song him college break....
3,e37c20cd-105b-4448-bf09-a61f5092c989,cbac3e24-e95f-4e1f-8e2e-4031ea902cd0,edecb2b8-1581-4150-a1b4-78a875bd4b58,4,2025-07-08 04:01:47,Authority choice career fly same by foreign. A...
4,b69a3015-800f-4eda-aa1e-c74307388813,cbac3e24-e95f-4e1f-8e2e-4031ea902cd0,e912dcf0-0256-4262-b00d-98978fff0aba,6,2024-04-23 10:30:45,During measure simply space boy. Affect play o...
...,...,...,...,...,...,...
176486,ef85df88-d5fd-49f8-a9b9-50f9c071e384,b3fe412a-9879-4c53-bd83-d4b1fde3d29a,8f858ceb-4255-4e4b-bf2f-a072d66072a0,8,2025-01-14 18:11:10,Capital election soldier movie explain. State ...
176487,40fd1f3f-08ae-4651-a457-2e812362ca34,b3fe412a-9879-4c53-bd83-d4b1fde3d29a,0d835bd5-4580-4c46-ab05-c804135a2a49,1,2024-11-12 11:09:34,Top live stuff probably physical. Note economi...
176488,9bc685fa-7aca-4f00-9633-43fb210f6d15,b3fe412a-9879-4c53-bd83-d4b1fde3d29a,09b2a920-8a11-4705-aae0-dc7294379f0a,10,2024-06-23 02:14:35,Word partner manager better give common less. ...
176489,d5deb240-feb7-466a-b05d-0217ed74d655,b3fe412a-9879-4c53-bd83-d4b1fde3d29a,43099d60-0454-47d5-8fe9-fbbec49c75fe,8,2025-09-20 23:05:29,Major action your now enter month scene. Sell ...


In [ ]:
raiting.to_sql('film_rating', engine, if_exists='append', index=False)

491

In [ ]:
def gen_list_title(cnt, g):
    return 'Top ' + str(cnt) + ' ' + g


list_cnt = 10
films_in_lst = random.choices([10, 15, 25], k=list_cnt)
list_guid = [uuid.uuid4() for i in range(list_cnt)]
list_title = [gen_list_title(films_in_lst[i], j) for i, j in enumerate(random.sample(list(genre[:, 0]), k=list_cnt))]

In [ ]:
films_lists = pd.DataFrame({
    'guid': list_guid,
    'title': list_title
})
films_lists

,guid,title
0,9f84c1a1-dd5f-4d49-8ded-1c5cb2cda91e,Top 10 War
1,eeb15f98-eecf-4e17-82f4-8e1128a2d668,Top 25 Romance
2,0fb522d7-b031-4be7-90fd-7f4992525efa,Top 10 Music
3,80b644de-97d9-4485-80fe-ef3d06d657a0,Top 15 Film-Noir
4,e77dd1a5-5e61-4a35-97c8-99670be38bfe,Top 15 Drama
5,23629345-0455-46bb-b4b7-89ba73257527,Top 15 Animation
6,7989f69d-1f5b-4c13-a130-a2572dd0619f,Top 15 Crime
7,336ba972-abb3-4e71-b99b-c43111d3abdd,Top 15 Family
8,92053be4-f7d3-447f-8859-12513e55acee,Top 25 Western
9,3e08cfed-57d3-4759-a797-6e27310365a7,Top 15 Thriller


In [ ]:
films_lists.to_sql('film_list', engine, if_exists='append', index=False)

10

In [ ]:
lst_ref = []
fl_film_ref = []
for i, j in enumerate(list_guid):
    lst_ref += [j] * films_in_lst[i]
    fl_film_ref += random.sample(films_uuid, k=films_in_lst[i])

In [ ]:
fl_films = pd.DataFrame({
    'list_guid': lst_ref,
    'film_guid': fl_film_ref
})
fl_films

,list_guid,film_guid
0,9f84c1a1-dd5f-4d49-8ded-1c5cb2cda91e,8c85c60e-f5ee-4a5e-bd9d-133c4f5674b4
1,9f84c1a1-dd5f-4d49-8ded-1c5cb2cda91e,793d2527-9421-4388-ae63-dc711e031154
2,9f84c1a1-dd5f-4d49-8ded-1c5cb2cda91e,6e8d4f40-9b16-404f-8b3f-987c04ce1884
3,9f84c1a1-dd5f-4d49-8ded-1c5cb2cda91e,16c8681f-a4a0-4417-9a47-7bdbd95c3a24
4,9f84c1a1-dd5f-4d49-8ded-1c5cb2cda91e,b4559806-25b8-4cf6-902a-9f22bf332a89
...,...,...
155,3e08cfed-57d3-4759-a797-6e27310365a7,666a3c90-b05c-46d0-b288-46ba753290d4
156,3e08cfed-57d3-4759-a797-6e27310365a7,2be32308-2371-44df-a46d-356b0e57d880
157,3e08cfed-57d3-4759-a797-6e27310365a7,7ac30ab6-f9d3-4283-a3bf-db14bb36353e
158,3e08cfed-57d3-4759-a797-6e27310365a7,21764846-2b20-48ca-b569-a7bf9c10736d


In [ ]:
fl_films.to_sql('fl_films', engine, if_exists='append', index=False)

In [ ]:
query = f'''SELECT pg_size_pretty(pg_database_size('filmsdb2'))'''
df_res = pd.read_sql(query, con=engine)
df_res

,pg_size_pretty
0,83 MB


## 4. Роли

#### Manager
```postgresql

CREATE ROLE manager;

GRANT USAGE ON SCHEMA public TO manager;

GRANT SELECT, INSERT, UPDATE ON films 
TO manager;

GRANT SELECT, INSERT, UPDATE ON persons 
TO manager;

GRANT SELECT, INSERT, UPDATE ON film_person 
TO manager;

GRANT SELECT, INSERT, UPDATE ON film_genre
TO manager;

GRANT SELECT, INSERT, UPDATE, DELETE on film_lists
to manager;

GRANT SELECT, INSERT, UPDATE, DELETE on fl_films
to manager;

```
#### Moderator

```postgresql
CREATE ROLE moderator;

GRANT USAGE ON SCHEMA public TO moderator;

GRANT SELECT, DELETE ON film_rating 
TO moderator;

GRANT SELECT, INSERT, UPDATE on users
to moderator;


CREATE USER moderator1 PASSWORD 'moder1'
IN ROLE moderator;
```
#### Reader

```postgresql
CREATE ROLE reader;
GRANT USAGE ON SCHEMA public TO reader;
GRANT SELECT ON ALL TABLES IN SCHEMA public
TO reader;

CREATE USER reader1 PASSWORD 'reader1'
IN ROLE reader;

--- Включение пользователя manager2 в группу reader 

GRANT reader to manager2 WITH INHERIT TRUE;
```


```postgresql
	select * from users
	limit 10;
```


![[manager.png]](images/manager.png)
```sql
delete from users where username = 'vlee';
```

```
filmsdb2=> delete from users where username = 'vlee';
ОШИБКА:  нет прав доступа к таблице users
```


```
filmsdb2=> create user manager3 password 'manager3' in role manager;
ОШИБКА:  нет прав для назначения членства в роли "manager"
ПОДРОБНОСТИ:  Только роли с привилегией ADMIN для роли "manager" могут назначать членов в ней.
```

```postgresql
ALTER ROLE manager2 with CREATEROLE;  
GRANT manager TO manager2 WITH ADMIN OPTION;
```

```
filmsdb2=> create user manager3 password 'manager3' in role manager;
CREATE ROLE
```

```
filmsdb2=# \du
                              List of roles
 Role name  |                         Attributes
------------+------------------------------------------------------------
 admin      | Superuser, Create role, Create DB, Bypass RLS
 manager    | Cannot login
 manager1   | Create role
 manager2   | Create role
 manager3   |
 moderator  | Cannot login
 moderator1 |
 postgres   | Superuser, Create role, Create DB, Replication, Bypass RLS
 reader     | Cannot login
 reader1    |
```

```sql
filmsdb2=# SELECT rolname FROM pg_roles WHERE rolcanlogin;
  rolname
------------
 postgres
 admin
 moderator1
 reader1
 manager1
 manager2
 manager3
(7 rows)
```

## View



Было создано представление для просмотра средней оценки фильма по пользователям

```postgresql
CREATE VIEW film_score AS
SELECT title, score, rates_cnt
FROM (
	SELECT film_guid, round(avg(rating), 2) as score, count(user_guid) AS rates_cnt
	FROM film_rating
	GROUP BY film_guid
    ) AS ratings
INNER JOIN films ON ratings.film_guid = films.guid
ORDER BY score DESC;
```

In [51]:
query = f'''SELECT *
            FROM film_score;
         '''
df_res = pd.read_sql(query, con=engine)
df_res

,title,score,rates_cnt
0,Birdman or (The Unexpected Virtue of Ignorance),6.31,126
1,The Killing Fields,6.28,133
2,The Silence of the Lambs,6.26,121
3,Hamlet,6.16,103
4,Sing Street,6.13,178
...,...,...,...
995,Tonari no Totoro,4.88,189
996,Andrei Rublev,4.87,111
997,A Christmas Story,4.85,100
998,Adams æbler,4.70,161
